# WSI - Ćwiczenie 7

*Autor: Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest implementacja narzędzia do wnioskowania przy użyciu sieci Bayesa i zademonstrowanie jego możliwości na następującym przykładzie:
- Mamy w domu zamontowany alarm przeciwwłamaniowy.
- Czasem alarm uruchamia się w przypadku niewielkich trzęsień ziemi.
- Umówiliśmy się z sąsiadami, aby dzwonili gdy usłyszą alarm.
    - Jan dzwoni do nas zawsze gdy usłyszy sygnał alarmu - czasem myli go nawet z innymi dźwiękami.
    - Magda dzwoni rzadziej - w domu lubi słuchać dość głośnej muzyki.
    
**Pytanie: Jak ocenimy szanse na to, że było włamanie w zależności od tego kto zadzwonił?**

#### Założenia

1. Wnioskowanie będzie stosowało algorytm MCMC z próbkowaniem Gibbsa,
2. Metoda/funkcja do wnioskowania będzie mieć 3 parametry wejściowe:
    1. Dowody - zaobserwowane wartości wybranych węzłów sieci,
    2. Zapytanie - określenie, dla której zmiennej chcemy wykonać obliczenia,
    3. Liczba iteracji algorytmu MCMC,
    
    oraz będzie zwracała zaktualizowaną tabelę prawdopodobieństw dla zmiennej z zapytania,
3. Można przyjąć założenie, że wszystkie zmienne losowe są binarne,
4. Eksperymenty powinny dotyczyć tego, jak zmieniają się wyniki wnioskowania i czas obliczeń wraz ze zwiększaniem liczby iteracji.


In [15]:
class BayesianNode:
    def __init__(self, name):
        self.name = name
        self.parents = []
        self.children = []
        self.value = None
    
    def add_parent(self, parent):
        self.parents.append(parent)
    
    def add_child(self, child):
        self.children.append(child)
    
    def set_value(self, value):
        self.value = value

In [16]:
class BayesianNetwork:
    def __init__(self, struct, prob):
        self.nodes = {}
        self.count = {}
        self.prob = prob
        for node, relation in struct.items():
            if node not in self.nodes:
                self.nodes[node] = BayesianNode(node)
            for parent in relation:
                if parent not in self.nodes:
                    self.nodes[parent] = BayesianNode(parent)
                self.nodes[node].add_parent(parent)
                self.nodes[parent].add_child(node)
    
    def set_node_value(self, node, value):
        self.nodes[node].set_value(value)
    
    def get_node_value(self, node):
        return self.nodes[node].value
    
    def get_node_parents(self, node):
        return self.nodes[node].parents
    
    def get_node_children(self, node):
        return self.nodes[node].children
    
    def get_node(self, node):
        return self.nodes[node]
    
    def add_node(self, name):
        if name not in self.nodes:
            self.nodes[name] = BayesianNode(name)
    
    def get_nodes(self):
        return self.nodes.keys()


In [17]:
import numpy as np

class Gibbs:
    def __init__(self, bn):
        self.bn = bn
    
    def ask(self, evidence, query, iters):
        for node, value in evidence.items():
            self.bn.set_node_value(node, value)
        for node in self.bn.get_nodes():
            if node not in evidence:
                self.bn.set_node_value(node, self.sample(self.bn.prob[node]))
        result = {node: {value: 0 for value in values} for node, values in query.items()}
        for _ in range(iters):
            for node in self.bn.get_nodes():
                if node not in evidence:
                    self.bn.set_node_value(node, self.sample(self.conditional_prob(node)))
            for node, values in query.items():
                for value in values:
                    if self.bn.get_node_value(node) == value:
                        result[node][value] += 1
        for node, values in result.items():
            total = sum(values.values())
            for value in values:
                result[node][value] /= total
        return result

    def conditional_prob(self, node):
        result = {}
        for value in ['True', 'False']:
            if len(self.bn.get_node_parents(node)) == 0:
                result[value] = self.bn.prob[node][value]
            else:
                parent_values = ','.join([str(self.bn.get_node_value(parent)) for parent in self.bn.get_node_parents(node)])
                result[value] = self.bn.prob[node][parent_values]
        return result
    
    def sample(self, prob):
        prob_sum = sum(prob.values())
        rand = np.random.uniform(0, prob_sum)
        all_prob = 0
        for value, _prob in prob.items():
            all_prob += _prob
            if rand <= all_prob:
                return value
        return value

In [22]:
network_struct = {
    'Burglary': [],
    'Earthquake': [],
    'Alarm': ['Burglary', 'Earthquake'],
    'JanCalls': ['Alarm'],
    'MagdaCalls': ['Alarm']
}

probabilities = {
    'Burglary': {
        'True': 0.01,
        'False': 0.99
    },
    'Earthquake': {
        'True': 0.02,
        'False': 0.98
    },
    'Alarm': {
        'True,True': 0.95,
        'True,False': 0.94,
        'False,True': 0.29,
        'False,False': 0.001
    },
    'JanCalls': {
        'True': 0.90,
        'False': 0.05
    },
    'MagdaCalls': {
        'True': 0.70,
        'False': 0.01
    }
}

bn = BayesianNetwork(network_struct, probabilities)
gibbs = Gibbs(bn)
print(gibbs.ask({'Alarm': 'True'}, {'Burglary': ['True', 'False'], 'Earthquake': ['True', 'False']}, 10000))

{'Burglary': {'True': 0.0095, 'False': 0.9905}, 'Earthquake': {'True': 0.0189, 'False': 0.9811}}
